In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning import LightningModule, Trainer
import numpy as np

# 定义LSTM模型
class LSTMModel(LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.save_hyperparameters()  # 保存超参数以供以后使用
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        # 确保目标张量y是Long类型
        y = y.long()
        loss = nn.CrossEntropyLoss()(y_hat, y)
        self.log('train_loss', loss)
        return loss    

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        val_loss = nn.CrossEntropyLoss()(y_hat, y)
        self.log('val_loss', val_loss, on_epoch=True, prog_bar=True, logger=True) # 记录验证损失
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def prepare_data(self):
        # 这里仅为示例，生成一些随机数据
        x_train = np.random.rand(100, 5, 10)  # (样本数, 序列长度, 特征数)
        y_train = np.random.randint(0, 2, 100)  # 随机标签
        self.train_dataset = TensorDataset(torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train))

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=32, shuffle=True)

# 实例化模型
input_size = 10  # 特征数
hidden_size = 50  # 隐藏层大小
num_layers = 2  # LSTM层数
output_size = 2  # 输出类别数

model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 准备数据
model.prepare_data()

# 实例化Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

early_stop_callback = EarlyStopping(
    monitor='val_loss',  # 指定监控指标
    min_delta=0.00,  # 指标提升的最小变化量
    patience=2,  # 没有提升时的等待轮数
    verbose=True,
    mode='min'
)

trainer = Trainer(
    max_epochs=100,  # 设置最大训练轮数
    callbacks=[early_stop_callback]
)

# 开始训练
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | lstm | LSTM   | 32.8 K
1 | fc   | Linear | 102   
--------------------------------
32.9 K    Trainable params
0         Non-trainable params
32.9 K    Total params
0.132     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 4/4 [00:00<00:00, 110.28it/s, v_num=52]

RuntimeError: Early stopping conditioned on metric `val_loss` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train_loss`